In [1]:
import cv2
import numpy as np
# import glob
# import os
import matplotlib.pyplot as plt

# Get max, int, min channels

In [2]:
def get_channels(image):
    b,g,r=cv2.split(image)
    b_mean=np.mean(b)
    g_mean=np.mean(g)
    r_mean=np.mean(r)
    
    max_val=max(b_mean,g_mean,r_mean)
    min_val=min(b_mean,g_mean,r_mean)
    
    if(max_val==b_mean):
        i_max=b
        print('blue')
        if(min_val==r_mean):
            i_min=r
        else:
            i_min=g
    elif(max_val==g_mean):
        i_max=g
        print('green')
        if(min_val==r_mean):
            i_min=r
        else:
            i_min=b
    else:
        i_max=r
        print('red')
        if(min_val==b_mean):
            i_min=b
        else:
            i_min=g
    
    if((max_val==g_mean and min_val==r_mean) or (min_val==g_mean and max_val==r_mean)):
        i_int=b
    elif((max_val==b_mean and min_val==r_mean) or (min_val==b_mean and max_val==r_mean)):
        i_int=g
    else:
        i_int=r
    
    return(i_max,i_int,i_min)

# Color Correction Weights

In [3]:
def get_attention_weights(i_max,i_int,i_min):
 
    att_int=i_max-i_int
    att_min=i_max-i_min
    att_max=np.absolute(i_max-i_int-i_min)
    
    return(att_max,att_int,att_min)
    #print(i_max,i_min,i_int)
    #print(np.mean(b))
    #print(b)

In [4]:
def attach_attention_weights(image,attention_weights,i_max,i_int,i_min):
    """
    Attach attention weights to color channels of an image.

    Args:
        image (numpy.ndarray): Input image in BGR format.
        attention_weights (list): List of attention weights for each channel.

    Returns:
        numpy.ndarray: Image with attention weights applied to color channels.
    """
    if len(attention_weights) != 3:
        raise ValueError("Attention weights should be a list of length 3 (for BGR channels).")

    # Split the input image into BGR channels
    b, g, r = cv2.split(image)
    

    # Apply attention weights to each channel
#     weighted_b = b * attention_weights[0]
#     weighted_g = g * attention_weights[1]
#     weighted_r = r * attention_weights[2]

    weighted_i_int=i_int+(attention_weights[1]*i_max)
    weighted_i_min=i_min+(attention_weights[2]*i_max)
    weighted_i_max=i_max#-(attention_weights[0]*intensities[0])
    
    if(np.array_equal(i_max,b)):
        weighted_b=weighted_i_max
        if(np.array_equal(i_int,r)):
            weighted_r=weighted_i_int
            weighted_g=weighted_i_min
        else:
            weighted_g=weighted_i_int
            weighted_r=weighted_i_min
    elif(np.array_equal(i_max,g)):
        weighted_g=weighted_i_max
        if(np.array_equal(i_int,r)):
            weighted_r=weighted_i_int
            weighted_b=weighted_i_min
        else:
            weighted_b=weighted_i_int
            weighted_r=weighted_i_min
    else:
        weighted_r=weighted_i_max
        if(np.array_equal(i_int,g)):
            weighted_g=weighted_i_int
            weighted_b=weighted_i_min
        else:
            weighted_b=weighted_i_int
            weighted_g=weighted_i_min
        
    

    # Merge the modified channels back into an image
    weighted_image = cv2.merge([weighted_b, weighted_g, weighted_r])
    
    #print(weighted_image.shape)
    
    Omax=np.max(weighted_image)
    Omin=np.min(weighted_image)
    
    Icr_max=Omin+(weighted_i_max-np.min(weighted_i_max))*((Omax-Omin)/(np.max(weighted_i_max)-np.min(weighted_i_max)))
    Icr_int=Omin+(weighted_i_int-np.min(weighted_i_int))*((Omax-Omin)/(np.max(weighted_i_int)-np.min(weighted_i_int)))
    Icr_min=Omin+(weighted_i_min-np.min(weighted_i_min))*((Omax-Omin)/(np.max(weighted_i_min)-np.min(weighted_i_min)))
    
    
    if(np.array_equal(i_max,b)):
        weighted_b=Icr_max
        if(np.array_equal(i_int,r)):
            weighted_r=Icr_int
            weighted_g=Icr_min
        else:
            weighted_g=Icr_int
            weighted_r=Icr_min
    elif(np.array_equal(i_max,g)):
        weighted_g=Icr_max
        if(np.array_equal(i_int,r)):
            weighted_r=Icr_int
            weighted_b=Icr_min
        else:
            weighted_b=Icr_int
            weighted_r=Icr_min
    else:
        weighted_r=Icr_max
        if(np.array_equal(i_int,g)):
            weighted_g=Icr_int
            weighted_b=Icr_min
        else:
            weighted_b=Icr_int
            weighted_g=Icr_min
    # Clip the values to ensure they are within the valid range [0, 255]
    
    corrected_image = cv2.merge([weighted_b, weighted_g, weighted_r])
    final_image = np.clip(corrected_image, 0, 255).astype(np.uint8)

    return (weighted_image,final_image)

# Remove noise

In [5]:
def remove_noise(image,kernel_size):
    """
    Remove noise from an input image using Gaussian smoothing.

    Args:
        input_image_path (str): Path to the input image.
        output_image_path (str): Path to save the denoised image.
        kernel_size (int): Size of the Gaussian kernel for smoothing.

    Returns:
        None
    """
    # Read the input image

    # Apply Gaussian smoothing to remove noise
    denoised_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

    # Save the denoised image
    #cv2.imwrite(output_image_path, denoised_image)
    return denoised_image

# Histogram maker

In [6]:
def histogram(image):

    # Convert the image to grayscale
    #grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    b,g,r=cv2.split(image)

    # Calculate the histogram of the grayscale image
    histblue = cv2.calcHist(b, [0], None, [256], [0, 256])
    histgreen = cv2.calcHist(g, [0], None, [256], [0, 256])
    histred = cv2.calcHist(r, [0], None, [256], [0, 256])
    
    figure, axis = plt.subplots(2, 2)
    # Plot the histogram
    axis[0,0].plot(histblue)
    #plt.show()
    axis[0,1].plot(histgreen)
    #plt.show()
    axis[1,0].plot(histred)
    plt.show()

# Rayleigh Distribution

In [7]:
def rayleigh_channel(icr,img,min_lim,max_lim,i_min,alpha):
    i_c_min=np.min(img)
    i_c_max=np.max(img)
    
    if(i_c_min>min_lim):
        O_min=i_c_min
    else:
        O_min=min_lim
        
    if(i_c_max>max_lim):
        O_max=i_c_max
    else:
        O_max=max_lim
    
    statement=(icr-i_min)*((O_max-O_min)/(np.max(i_min)-np.min(i_min)))
    
    internal=(O_min+statement)/(2*alpha*alpha)
    exponent=np.exp(-1*internal*internal)
    
    external=statement/(alpha*alpha)
    
    irw=external*exponent
    
    return irw    

In [8]:
def rayleigh_dist(icr,image,i_min,alpha):
    b,g,r=cv2.split(image)
    
    minimum_bgr=min(np.min(b),np.min(g),np.min(r))
    
    three_point_five=0.035*minimum_bgr
    ninesix_point_five=0.965*minimum_bgr
    
    bcr,gcr,rcr=cv2.split(icr)
    
#     mini=np.min(i_min)
#     maxi=np.max(i_min)
    
#     if(mini>three_point_five):
#         O_min=mini
#     else:
#         O_min=three_point_five
        
#     if(maxi<ninesix_point_five):
#         O_max=maxi
#     else:
#         O_max=ninesix_point_five
    
#     #internal=np.power(((O_min+(icr-i_min)*((O_max-O_min)/(np.max(i_min)-np.min(i_min))))/(2*alpha*alpha)),2)
    
#     print(icr.shape)
#     print(i_min.shape)
    
#     statement=(icr-i_min)*((O_max-O_min)/(np.max(i_min)-np.min(i_min)))
    
#     internal=(O_min+statement)/(2*alpha*alpha)
#     exponent=np.exp(-1*internal*internal)
    
#     external=statement/(alpha*alpha)
    
#     irw=external*exponent

    b_rw=rayleigh_channel(bcr,b,three_point_five,ninesix_point_five,i_min,alpha)
    g_rw=rayleigh_channel(gcr,g,three_point_five,ninesix_point_five,i_min,alpha)
    r_rw=rayleigh_channel(rcr,r,three_point_five,ninesix_point_five,i_min,alpha)
    
    irw=cv2.merge([b_rw,g_rw,r_rw])
    return irw    

# Global contrast

In [9]:
def forebackinit(c):
    m=len(c)
    n=len(c[0])
    b = np.zeros_like(c)
    f = np.zeros_like(c)
    i_max=c.max()
    i_min=c.min()
    loss=1
    num1=0
    num2=0
    s1=0
    s2=0
    i_init=(i_min+i_max)/2
    while loss>=0.5:
        for i in range(1,m):
            for j in range(1,n):
                if c[i,j]>i_init:
                    num1=num1+1
                    s1=s1+c[i,j]
                else:
                    num2=num2+1
                    s2=s2+c[i,j]
        mean_s1=s1/num1
        mean_s2=s2/num2
        i_new=(mean_s1+mean_s2)/2
        loss=abs(i_init-i_new)
        i_init=i_new
        if loss>=0.5:
            num1=0
            num2=0
            s1=0
            s2=0
    i_opt=i_init
    for i in range(1,m):
        for j in range(1,n):
            if c[i,j]<i_opt:
                b[i,j]=c[i,j]
            else:
                f[i,j]=c[i,j]
    
    return(b,f,i_opt)

In [10]:
def combine_images(image1, image2, threshold):

  # Convert the images to grayscale.
#     gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
#     gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

      # Threshold the images.
    thresh1 = cv2.threshold(image1, threshold, 255, cv2.THRESH_BINARY)[1]
    thresh2 = cv2.threshold(image2, threshold, 255, cv2.THRESH_BINARY_INV)[1]

      # Combine the thresholded images.
    combined_image = cv2.bitwise_or(thresh1, thresh2)

      # Return the combined image.


    return combined_image

In [11]:
def global_image(img):
    blue,green,red = cv2.split(img)
    bb,bf,bt=forebackinit(blue)
    gb,gf,gt=forebackinit(green)
    rb,rf,rt=forebackinit(red)
    
    t=0.067
    combined_b=fuse_images(bb,bf,t)
    combined_g=fuse_images(gb,gf,t)
    combined_r=fuse_images(rb,rf,t)
    
    i_b=cv2.merge([bb,gb,rb])
    i_f=cv2.merge([bf,gf,rf])
    result=cv2.merge([combined_b,combined_g,combined_r])
    
    return (result,i_b,i_f)
#     cv2.imshow("Background", i_b)
#     cv2.imshow("Foreground",i_f)

# MSF

In [12]:
def fuse_images(image1, image2, alpha):
    if image1.shape != image2.shape:
        raise ValueError("The images must be the same size.")
    beta = 1.0 - alpha
    fused_image = cv2.addWeighted(image1, beta, image2, alpha, 0,dtype=cv2.CV_32F).astype(np.uint8)
    return fused_image    

# MAIN

In [101]:
# Example usage:
if __name__ == "__main__":
    # Load an example image (replace with your image path)
    
    #image_path = r"C:\Users\Megha\Desktop\IP\Project\Implementation\images\lol_dataset\our485\low\9.png"
    image_path=r"C:\Users\Megha\Desktop\IP\Project\Implementation\images\lol_dataset\our485\low\131.png"
    #image_path = r"C:\Users\Megha\Desktop\IP\Project\underwater0.jpeg"
    image = cv2.imread(image_path)
    
    #histogram(image)
    
    i_max,i_int,i_min=get_channels(image)
    
    # Define attention weights for BGR channels (adjust these values as needed)
    attention_weights = get_attention_weights(i_max,i_int,i_min) #[1.2, 0.8, 1.0]
    

    #Apply attention weights to the image
    weighted_image,corrected_image = attach_attention_weights(image,attention_weights,i_max,i_int,i_min)    
    
    global_img,background,foreground=global_image(corrected_image)
    
    
#     kernel_size = 5
    
#     weighted_nonoise=remove_noise(weighted_image, kernel_size)
#     corrected_nonoise=remove_noise(corrected_image, kernel_size)
    
#     print("ICR")
#     histogram(corrected_nonoise)
#     print("WI")
#     histogram(weighted_nonoise)
    
    rayleigh_image=rayleigh_dist(corrected_image,image,i_min,alpha=10.0)
    fused_image=fuse_images(background,rayleigh_image,0.05)
    
    # Display the original and weighted images
#     cv2.imshow("Color Corrected Image", corrected_image)
#     cv2.imshow("Rayleigh Image", rayleigh_image)
    cv2.imshow("Fused Image", fused_image)
#     cv2.imshow("Background Image",background)
#     cv2.imshow("Foregorund Image",foreground)
#     cv2.imshow("Rayleigh Image",rayleigh_image)
#     cv2.imshow("Corrected Image",corrected_image)
    
    
#     cv2.imshow("Weighted Image", weighted_nonoise)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

red


# Metrics

https://stackoverflow.com/questions/13875989/comparing-image-in-url-to-image-in-filesystem-in-python/13884956#13884956

## BRISQUE

In [40]:
from brisque import BRISQUE

obj = BRISQUE(url=False)
print(obj.score(image))
print(obj.score(weighted_image))
print(obj.score(rayleigh_image))
print(obj.score(fused_image))
print(obj.score(background))
print(obj.score(foreground))

26.34065580884871
48.73153243483941
50.22642512638296
39.692637804484974
41.37039440762416
56.650914671362756


## Average Gradient

In [41]:
def calculate_average_gradient_metric(image):
    # Convert the image to grayscale.
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Compute the gradients of the image.
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    # Calculate the magnitude of the gradients.
    gradient_magnitude = cv2.magnitude(sobelx, sobely)
    # Calculate the average gradient metric.
    average_gradient_metric = np.mean(gradient_magnitude)
    # Return the average gradient metric.
    return average_gradient_metric

In [48]:
print(calculate_average_gradient_metric(fused_image))
print(calculate_average_gradient_metric(corrected_image))
print(calculate_average_gradient_metric(image))

51.44626774090773
94.45831800720772
8.235941968795236


## PCQI

In [51]:
def extract_patch(image, patch_size):
    # Calculate the center of the patch.
    patch_center = (patch_size[0] // 2, patch_size[1] // 2)
    # Extract the patch from the image.
    patch = image[patch_center[0] - patch_size[0] // 2:patch_center[0] + patch_size[0] // 2,
                patch_center[1] - patch_size[1] // 2:patch_center[1] + patch_size[1] // 2,
                :]
    # Return the patch.
    return patch

In [54]:
def calculate_pcqi(image):
    # Split the image into patches.
    patches = extract_patch(image, (32, 32))
    # Calculate the contrast of each patch.
    patch_contrasts = []
    for patch in patches:
        patch_contrast = cv2.Laplacian(patch, cv2.CV_64F).var()
        patch_contrasts.append(patch_contrast)
    # Calculate the average patch contrast.
    average_patch_contrast = np.mean(patch_contrasts)
    
    # Calculate the PCQI.
    pcqi = average_patch_contrast / 255.0
    # Return the PCQI.
    return pcqi

In [56]:
print(calculate_pcqi(fused_image))
print(calculate_pcqi(image))

13.335035123804294
0.2322793798508987


## NIQE

In [57]:
import scipy.signal
import scipy.special

In [93]:
def calculate_mscns(image):
    # Calculate the local mean and standard deviation of the image.
    local_mean = scipy.signal.convolve2d(image, np.ones((3, 3)), 'same') / 9.0
    local_std = np.sqrt(scipy.signal.convolve2d((image - local_mean)**2, np.ones((3, 3)), 'same') / 9.0)
    
    zero_values = np.where(local_std == 0)
    
    # Replace zero values with a small positive value.
    local_std[zero_values] = 1e-6
    # Calculate the MSCN coefficients.
    
    mscns = (image - local_mean) / local_std
    # Return the MSCN coefficients.
    return mscns

In [94]:
def calculate_ggd_parameters(mscn_coefficients):
    # Calculate the mean and standard deviation of the MSCN coefficients.
    mean = np.mean(mscn_coefficients)
    std = np.std(mscn_coefficients)
    # Calculate the shape parameter of the GGD.
    alpha = scipy.special.gamma((11.0 / 6.0)) / scipy.special.gamma((7.0 / 6.0)) * ((std / mean)**2)
    # Calculate the GGD parameters.
    ggd_parameters = [mean, std, alpha]
    # Return the GGD parameters.
    return ggd_parameters

In [95]:
def calculate_niqe_score(ggd_parameters):
    # Calculate the NIQE score.
    niqe_score = ggd_parameters[0] * np.sqrt(ggd_parameters[2]) + 0.15 * ggd_parameters[1]
    # Return the NIQE score.
    return niqe_score

In [98]:
def calculate_niqe(image):
    # Convert the image to grayscale.
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Calculate the locally normalized luminescence (MSCN) coefficients.
    mscns = calculate_mscns(gray)
    # Calculate the generalized Gaussian distribution (GGD) parameters.
    ggd_parameters = calculate_ggd_parameters(mscns)
    # Calculate the NIQE.
    niqe = calculate_niqe_score(ggd_parameters)
    # Return the NIQE.
    return abs(niqe)

In [99]:
print(calculate_niqe(fused_image))
print(calculate_niqe(image))

0.8328188976939206
1.1450653636047683


## Information Entropy

In [91]:
def calculate_image_entropy(image):
    # Check for zero values in the image.
#     zero_values = np.where(image == 0)
    
#     # Replace zero values with a small positive value.
#     image[zero_values] = 1e-6
    
    # Calculate the histogram of the image.
    histogram = np.histogram(image, bins=256)[0]
    
    # Calculate the probability of each pixel value.
    p_pixel_values = histogram / histogram.sum()
    
    zero_values = np.where(p_pixel_values == 0)
    
    # Replace zero values with a small positive value.
    p_pixel_values[zero_values] = 1e-6
    
    # Calculate the information entropy of the image.
    information_entropy = -np.sum(p_pixel_values * np.log2(p_pixel_values))
    # Return the information entropy of the image.
    return information_entropy

In [92]:
print(calculate_image_entropy(fused_image))
print(calculate_image_entropy(image))

3.9573788706891264
4.901051396781516
